# Project: ICD-AIS conversion using Deep Learning utilizing ICD10

This script translates checks the translations against the actual AIS codes that were recorded in NTDB.

## Setup

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

#### Files

In [2]:
# translations
aaam_ais_file = "../Results/Translations/ais_aaam_map.csv"
icdpic_ais_file = "../Results/Translations/test_ais_icdpic.csv"
nmt_ais_file = "../Results/Translations/nmt_ais.csv"

# observed ais codes
test_ais_file = "../Results/Translations/ais_obs.csv"

# AIS codes
ais_codes_file = "../Tools/AIS08_codes.csv"

# output for results
results_file = "../Results/ais_results.csv"

## Load data

In [3]:
# Load result files
aaam = pd.read_csv(aaam_ais_file)
icdpic = pd.read_csv(icdpic_ais_file)
nmt = pd.read_csv(nmt_ais_file)

In [4]:
# read test observations
test_obs = pd.read_csv(test_ais_file)

## Function to match AIS codes from two list
This function will return the best estimate for matching two lists of AIS codes.

In [5]:
def match_ais(ais1, ais2):
    '''
    This function matches AIS codes from two lists with proximation.  The overarching concept
    is to match codes based on 1) exact matches, 2) same body region-same severity, 
    3) same body region-different severity, 4) different body region-same severity, 
    5) remaining codes are paired in decreasing severity, 6) unmatched codes are then added.
    
    Parameters:
        ais1 - list of AIS codes
        ais2 - list of AIS codes
    Returns:
        Pandas dataframe with matched codes
    '''
    
    # df for matches
    matches = pd.DataFrame(columns={'codes1','code2'})
    
    # df for data
    df1 = pd.DataFrame({'ais_code':ais1, 'curr_prec':ais1}).sort_values('ais_code')
    df2 = pd.DataFrame({'ais_code':ais2, 'curr_prec':ais2}).sort_values('ais_code')

    # determine body region
    df1['region'] = np.floor(df1.ais_code/100000)
    df2['region'] = np.floor(df2.ais_code/100000)
    
    # determine severity
    df1['severity'] = np.round(df1.curr_prec%1 * 10)
    df2['severity'] = np.round(df2.curr_prec%1 * 10)

    # move severity to the 2nd MSD
    df1['curr_prec'] = np.floor(df1.region * 1000000 + df1.severity * 100000 + df1.ais_code%100000)
    df2['curr_prec'] = np.floor(df2.region * 1000000 + df2.severity * 100000 + df2.ais_code%100000)    
    
    # matching steps #1-3, loop through all digits
    for digit in range(0,7):

        # loop through ais1 codes
        for i in range(0,len(df1)):

            # loop through ais2 codes
            for j in range(0,len(df2)):

                # check for match
                if df1.curr_prec[i] == df2.curr_prec[j]:

                    # match found, add to match list
                    matches = matches.append({'code1':df1.ais_code[i], 'code2':df2.ais_code[j]}, ignore_index=True)

                    # remove rows with matched codes from dfs
                    df1 = df1.drop(index=i, axis=1)
                    df2 = df2.drop(index=j, axis=1).reset_index(drop=True)

                    # stop searching for code
                    break

        # decrease precision, on first loop this removes severity
        df1['curr_prec'] = np.floor(df1.curr_prec/10)
        df2['curr_prec'] = np.floor(df2.curr_prec/10)

        # reset index for list1
        df1 = df1.reset_index(drop=True) 

    # matching step #4 - find same severity in different body regions    
        
    # assign severity to current precision
    df1['curr_prec'] = df1.severity
    df2['curr_prec'] = df2.severity
    
    # sort based on codes
    df1 = df1.sort_values('ais_code').reset_index(drop=True)
    df2 = df2.sort_values('ais_code').reset_index(drop=True)
    
    #loop through all unmatched codes in list 1
    for i in range(0,len(df1)):
        
        # loop through unmatched codes in list 2
        for j in range(0,len(df2)):
                
            # check for match
            if df1.curr_prec[i] == df2.curr_prec[j]:

                # match found, add to match list
                matches = matches.append({'code1':df1.ais_code[i], 'code2':df2.ais_code[j]}, ignore_index=True)

                # remove rows with matched codes from dfs
                df1 = df1.drop(index=i, axis=1)
                df2 = df2.drop(index=j, axis=1).reset_index(drop=True)

                # stop searching for code
                break
                
    # matching step #5-6, sequentially assign unmatched codes from based on decreasing AIS severity

    # arrange in decreasing severity
    df1 = df1.sort_values('curr_prec', ascending=False).reset_index(drop=True)
    df2 = df2.sort_values('curr_prec', ascending=False).reset_index(drop=True)
  
    #loop through all unmatched codes in list 1
    for i in range(0,len(df1)):

        # check if list 2 still has any numbers
        if len(df2) > 0:

            # add to match list
            matches = matches.append({'code1':df1.ais_code[i], 'code2':df2.ais_code[0]}, ignore_index=True)

            # remove rows with matched codes from df
            df2 = df2.drop(index=0, axis=1).reset_index(drop=True)

        # else no more codes in list 2
        else:

            # add to match list
            matches = matches.append({'code1':df1.ais_code[i], 'code2':None}, ignore_index=True)

        # remove code from list 1
        df1 = df1.drop(index=i, axis=1)

    # assign unmatched to any remaining codes list 2
    for j in range(0,len(df2)):

        # add to match list
        matches = matches.append({'code1':None, 'code2':df2.ais_code[j]}, ignore_index=True)
        
    # remove values that are undefined
    matches['code1'] = matches.code1.apply(lambda x: None if x<=0 else x)
    matches['code2'] = matches.code2.apply(lambda x: None if x<=0 else x)

    # arrange by decrease AIS severity
    matches['severity1'] = matches.code1%1
    matches = matches.sort_values('code1').sort_values('severity1',ascending=False)[['code1','code2']].reset_index(drop=True)

    return matches

## Function to calculate ISS

In [6]:
def calc_ISS(codes_list, region_list, NISS=False):
    '''
    This function accepts a list of AIS codes and returns the ISS.  This is based
    on the six body regions method.
    
    Parameters:
        ais_codes - list of AIS codes
        NISS - True if the new injury severity score method should be used
        
    Returns:
        ISS or NISS
    '''
    
    # make dataframe from lists
    codes_df = pd.DataFrame({'code':codes_list, 'region':region_list}).fillna(0)
    
    #print(codes_df)
    
    # get severity and severity squared
    codes_df['severity'] = ((codes_df.code*10)%10).astype(int)
    codes_df['severity_sq'] = np.square(codes_df.severity)
       
    # check if any severity greater than 7, then unknown
    if (codes_df.severity>=7).any():
        # IS is unknown
        IS = np.nan

    # else check if any severity is 6, then automatic 75
    elif (codes_df.severity==6).any():
        # IS is max (75)
        IS = 75
        
    # calculate injury severity
    else:
        # check if using NISS, highest severity in any region
        if NISS:
            # get 3 highest severity codes
            codes_df = codes_df.sort_values('severity', ascending=False).reset_index(drop=True).head(3)

            # calculate injury severity
            IS = sum(codes_df.severity_sq)
            
        # else using ISS, highest severity in different body regions
        else:
            # get highest severity codes in different body regions
            codes_df = codes_df.sort_values('severity', ascending=False).groupby('region').head(1).reset_index(drop=True)
            
            # get 3 highest severity codes
            codes_df = codes_df.sort_values('severity', ascending=False).reset_index(drop=True).head(3)
            
            # calculate injury severity
            IS = sum(codes_df.severity_sq)
        
    return IS

## Function to evaluate matches

In [7]:
def eval_matches(codes_df, region='chapter'):
    '''
    This function determines the level of match  between paired
    lists of AIS codes.  

    Parameter:
        codes_df - dataframe with matched list of AIS codes, two columns of AIS codes
        
    Returns:
        Dataframe with match level totals for injury lists.  The levels are:
            exact - number of exact matches
            same_reg_same_sev - same body region, same severity, but not exact match
            same_reg_diff_sev - same body region, different severity 
            diff_reg_same_sev - same body region, different severity 
            diff_reg_diff_sev - different body region, different severity, but matched
            unmatched_obs - number of unmatched observed codes
            unmatched_pred - number of unmatched predicted codes
            region - type of region to use for calculation, options:
                'chapter' = AIS manual chapter [1st digit of code]
                'iss' = regions used to calculate ISS
    '''
    
    # make sure column names are correct
    codes_df = codes_df.rename(columns={codes_df.columns[0]:'obs',codes_df.columns[1]:'pred'})
    
    # fill in NaN with 0
    codes_df = codes_df.fillna(0)
    
    # add region to codes
    if region == 'chapter':
        # if using chapter for region get MSD for region
        codes_df['reg_obs'] = np.floor(codes_df.obs/100000).astype(int)
        codes_df['reg_pred'] = np.floor(codes_df.pred/100000).astype(int)
        
    else:
        # if using ISS region, get 2nd MSD for region
        codes_df['reg_obs'] = np.floor((codes_df.obs/10000)%10).astype(int)
        codes_df['reg_pred'] = np.floor((codes_df.pred/10000)%10).astype(int)        
    
    # get severity and severity squared
    codes_df['sev_obs'] = ((codes_df.obs*10)%10).astype(int)
    codes_df['sev_pred'] = ((codes_df.pred*10)%10).astype(int)
    
    # evaluate for exact matches
    codes_df['exact'] = codes_df.apply(lambda x: 1 if x['obs']==x['pred'] else 0, axis=1)
    
    # evaluate for same region, same severity, but not exact match
    codes_df['same_reg_same_sev'] = codes_df.apply(lambda x: 1 if ((x['exact']==0) & \
                                                                   (x['reg_obs']==x['reg_pred']) & \
                                                                   (x['sev_obs']==x['sev_pred'])) else 0, axis=1)
    
    # evaluate for same region, different severity
    codes_df['same_reg_diff_sev'] = codes_df.apply(lambda x: 1 if ((x['reg_obs']==x['reg_pred']) & \
                                                                   (x['sev_obs']!=x['sev_pred'])) else 0, axis=1)
    
    # evaluate for different region, same severity
    codes_df['diff_reg_same_sev'] = codes_df.apply(lambda x: 1 if ((x['reg_obs']!=x['reg_pred']) & \
                                                                   (x['sev_obs']==x['sev_pred'])) else 0, axis=1)
    
    # evaluate for different region, different severity, but not completely unmatched
    codes_df['diff_reg_diff_sev'] = codes_df.apply(lambda x: 1 if ((x['reg_obs']!=x['reg_pred']) & \
                                                                   (x['sev_obs']!=x['sev_pred']) & \
                                                                   (x['obs']!=0) & (x['pred']!=0)) else 0, axis=1)
    
    # evaluate for unmatched codes
    codes_df['unmatched_obs'] = codes_df.apply(lambda x: 1 if x['pred']==0 else 0, axis=1)    
    codes_df['unmatched_pred'] = codes_df.apply(lambda x: 1 if x['obs']==0 else 0, axis=1) 
        
    return codes_df

## Get AIS code match statistics

In [8]:
def add_iss_region_df(df):
    
    return df.aiscode - (np.floor(df.aiscode/10000)%10)*10000 + df.region*10000

In [9]:
def eval_matches_df(obs, pred, convert_region_obs, convert_region_pred, region='chapter', col_suff=''):
    '''
    This function takes a two dataframes with injuries list.  It matches injuries 
    based on the patient key and then evaluates the quality of the matches.

    Parameter:
        obs - dataframe with list of observed AIS codes (must contain 'inc_key' and 'aiscode')
        pred - dataframe with list of predicted AIS codes (must contain 'inc_key' and 'aiscode')
        convert_region_obs - convert the 2nd MSD to the region for observations
        convert_region_obs - convert the 2nd MSD to the region for predictins
        region - 'chapter' for AIS manual chapter regions, 'iss' for ISS regions
        
    Returns:
        Dataframe with a summary of the match level totals for all patients.  The levels are:
            exact - number of exact matches
            same_reg_same_sev - same body region, same severity, but not exact match
            same_reg_diff_sev - same body region, different severity 
            diff_reg_same_sev - same body region, different severity 
            diff_reg_diff_sev - different body region, different severity, but matched
            unmatched_obs - number of unmatched observed codes
            unmatched_pred - number of unmatched predicted codes
            convert_region_obs - True if the 2nd MSD should be replaced with the region
            convert_region_pred - True if the 2nd MSD should be replaced with the region
            region - type of region to use for calculation, options:
                'chapter' = AIS manual chapter [1st digit of code]
                'iss' = regions used to calculate ISS
            col_suff - suffix for columns with results
    '''
    
    # get all keys from prediction and observations
    keys = np.unique(np.concatenate([test_obs['inc_key'], aaam['inc_key']]))
    
    # loop through all keys
    for i,key in enumerate(keys):
        
        #print(i)
        
        # get aiscodes from observations
        if convert_region_obs==True:
            # obtain aiscodes and add region to 2nd MSB
            obs_codes = add_iss_region_df(obs[obs.inc_key==key].reset_index(drop=True))
        
        else:
            # obtain aiscodes without modification
            obs_codes = obs[obs.inc_key==key].reset_index(drop=True).aiscode
            
        # get aiscodes from predictions
        if convert_region_pred==True:
            # obtain aiscodes and add region to 2nd MSB
            pred_codes = add_iss_region_df(pred[pred.inc_key==key].reset_index(drop=True))

        else:
            # obtain aiscodes without modification
            pred_codes = pred[pred.inc_key==key].reset_index(drop=True).aiscode
        
        # match observed and predicted code
        matched_list = match_ais(obs_codes, pred_codes)
        
        # evaluate level of matches
        match_results = eval_matches(matched_list, region)
        
        # sum results and select for necessary columns
        results_sum = pd.DataFrame(match_results.sum(axis=0)).T[['exact','same_reg_same_sev',
                                                                 'same_reg_diff_sev','diff_reg_same_sev',
                                                                 'diff_reg_diff_sev','unmatched_obs',
                                                                 'unmatched_pred']]
        # add key
        results_sum['inc_key'] = key
                
        # store in dataframe
        if i==0:
            results = results_sum.copy()
        else:
            results = results.append(results_sum).reset_index(drop=True)
                   
        #if i>=100:
        #    break
            
        if (i%1000)==0:
            print(i)
    
    # add suffix for column names, except inc_key
    results = results.add_suffix(col_suff).rename(columns={'inc_key'+col_suff: 'inc_key'})
    
    return results

## Function to determine ISS from dataframe

In [10]:
def df_iss(df):
    '''
    Calculates Injury Severity Score from a datafame.  The dataframe
    should have one injury per row and should have the following columns:
        inc_key - unique patient key
        aiscode - fully AIS code
        region - ISS region of injury
    
    Paramaters:
    df - datafame
    
    Returns:
    ISS
    '''
    
    # list for ISS results
    iss=[]
    
    # dataframe for pt data
    df_pt = df[['inc_key']].drop_duplicates().reset_index(drop=True)
    
    # get lists of observe ais codes
    df_pt['ais']=df.groupby('inc_key')['aiscode'].apply(list).values
    df_pt['region']=df.groupby('inc_key')['region'].apply(list).values
    
    # iterrate through each row
    for i,row in df_pt.iterrows():
        iss.append(calc_ISS(row.ais, row.region))
        
    return iss

In [11]:
def df_mais(df):
    '''
    Calculates Maximum Abbreviated Injury Score from a datafame.  The dataframe
    should have one injury per row and should have the following columns:
        inc_key - unique patient key
        severity - AIS severity as an integer.
    
    Paramaters:
    df - datafame
    
    Returns:
    MAIS
    '''
    
    # group by key and get max severity
    return df.groupby('inc_key')['severity'].max().values   

## Create results dataframe

In [12]:
# make dataframe for results
res = test_obs[['inc_key']].drop_duplicates().reset_index(drop=True)

In [13]:
len(icdpic.inc_key.unique())

10000

## Calc ISS

In [14]:
%%time
# iss for observations, nmt, AAAM map, and ICDPICR
res['iss_obs'] = df_iss(test_obs)
res['iss_nmt'] = df_iss(nmt)
res['iss_aaam'] = df_iss(aaam)
res['iss_icdpic'] = df_iss(icdpic)

CPU times: user 1min 40s, sys: 127 ms, total: 1min 40s
Wall time: 1min 40s


In [15]:
# check if prediction match observations exact iss
res['iss_correct_nmt'] = np.where(res['iss_obs']==res['iss_nmt'],1,0)
res['iss_correct_aaam'] = np.where(res['iss_obs']==res['iss_aaam'],1,0)
res['iss_correct_icdpic'] = np.where(res['iss_obs']==res['iss_icdpic'],1,0)

In [16]:
res.mean()

inc_key               1.900334e+11
iss_obs               8.869521e+00
iss_nmt               8.590475e+00
iss_aaam              7.893000e+00
iss_icdpic            6.817600e+00
iss_correct_nmt       7.779000e-01
iss_correct_aaam      6.894000e-01
iss_correct_icdpic    1.772000e-01
dtype: float64

## Calc ISS>=16

In [17]:
def iss16 (iss): 
    return 1 if iss >=16 else 0

In [18]:
# calculate columns for iss>=16
res['iss16_obs'] = res.iss_obs.apply(iss16)
res['iss16_nmt'] = res.iss_nmt.apply(iss16)
res['iss16_aaam'] = res.iss_aaam.apply(iss16)
res['iss16_icdpic'] = res.iss_icdpic.apply(iss16)

In [19]:
# check if prediction match observations iss>=16
res['iss16_correct_nmt'] = np.where(res['iss16_obs']==res['iss16_nmt'],1,0)
res['iss16_correct_aaam'] = np.where(res['iss16_obs']==res['iss16_aaam'],1,0)
res['iss16_correct_icdpic'] = np.where(res['iss16_obs']==res['iss16_icdpic'],1,0)

In [20]:
res.filter(regex='iss16_correct').mean()

iss16_correct_nmt       0.9368
iss16_correct_aaam      0.9132
iss16_correct_icdpic    0.8732
dtype: float64

## Calc MAIS

In [21]:
res['mais_obs'] = df_mais(test_obs)
res['mais_nmt'] = df_mais(nmt)
res['mais_aaam'] = df_mais(aaam)
res['mais_icdpic'] = df_mais(icdpic)

In [22]:
res.filter(regex='mais').mean()

mais_obs       2.59862
mais_nmt       2.58630
mais_aaam      2.27750
mais_icdpic    2.03050
dtype: float64

## Calc MAIS >=3

In [23]:
def mais3 (mais): 
    return 1 if mais >=3 else 0

In [24]:
# calculate columns for mais>=3
res['mais3_obs'] = res.mais_obs.apply(mais3)
res['mais3_nmt'] = res.mais_nmt.apply(mais3)
res['mais3_aaam'] = res.mais_aaam.apply(mais3)
res['mais3_icdpic'] = res.mais_icdpic.apply(mais3)

In [25]:
# check if prediction match observations mais>=3
res['mais3_correct_nmt'] = np.where(res['mais3_obs']==res['mais3_nmt'],1,0)
res['mais3_correct_aaam'] = np.where(res['mais3_obs']==res['mais3_aaam'],1,0)
res['mais3_correct_icdpic'] = np.where(res['mais3_obs']==res['mais3_icdpic'],1,0)

In [26]:
res.filter(regex='mais3').mean()

mais3_obs               0.3296
mais3_nmt               0.2990
mais3_aaam              0.3800
mais3_icdpic            0.3162
mais3_correct_nmt       0.9126
mais3_correct_aaam      0.8450
mais3_correct_icdpic    0.6880
dtype: float64

## Calc MAIS >=2

In [27]:
def mais2 (mais): 
    return 1 if mais >=2 else 0

In [28]:
# calculate columns for mais>=3
res['mais2_obs'] = res.mais_obs.apply(mais2)
res['mais2_nmt'] = res.mais_nmt.apply(mais2)
res['mais2_aaam'] = res.mais_aaam.apply(mais2)
res['mais2_icdpic'] = res.mais_icdpic.apply(mais2)

In [29]:
# check if prediction match observations mais>=3
res['mais2_correct_nmt'] = np.where(res['mais2_obs']==res['mais2_nmt'],1,0)
res['mais2_correct_aaam'] = np.where(res['mais2_obs']==res['mais2_aaam'],1,0)
res['mais2_correct_icdpic'] = np.where(res['mais2_obs']==res['mais2_icdpic'],1,0)

In [30]:
res.filter(regex='mais2').mean()

mais2_obs               0.6154
mais2_nmt               0.6102
mais2_aaam              0.8277
mais2_icdpic            0.5601
mais2_correct_nmt       0.9564
mais2_correct_aaam      0.7481
mais2_correct_icdpic    0.7333
dtype: float64

## Calc match quality

In [31]:
%%time
# get match quality
match_nmt = eval_matches_df(test_obs, nmt, convert_region_obs=True, convert_region_pred=True, region='iss', col_suff='_nmt')

# add to results
res = res.merge(match_nmt, on='inc_key', how='left')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
CPU times: user 5min 12s, sys: 482 ms, total: 5min 12s
Wall time: 5min 13s


In [32]:
%%time
# get match quality
match_aaam = eval_matches_df(test_obs, aaam, convert_region_obs=True, convert_region_pred=False, region='iss', col_suff='_aaam')

# add to results
res = res.merge(match_aaam, on='inc_key', how='left')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
CPU times: user 5min 14s, sys: 660 ms, total: 5min 15s
Wall time: 5min 15s


In [33]:
%%time
# get match quality
match_icdpic = eval_matches_df(test_obs, icdpic, convert_region_obs=True, convert_region_pred=False, region='iss', col_suff='_icdpic')

# add to results
res = res.merge(match_icdpic, on='inc_key', how='left')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
CPU times: user 5min 21s, sys: 226 ms, total: 5min 21s
Wall time: 5min 22s


In [34]:
res.isna().sum()

inc_key                       0
iss_obs                     236
iss_nmt                     257
iss_aaam                      0
iss_icdpic                    0
iss_correct_nmt               0
iss_correct_aaam              0
iss_correct_icdpic            0
iss16_obs                     0
iss16_nmt                     0
iss16_aaam                    0
iss16_icdpic                  0
iss16_correct_nmt             0
iss16_correct_aaam            0
iss16_correct_icdpic          0
mais_obs                      2
mais_nmt                      0
mais_aaam                     0
mais_icdpic                   0
mais3_obs                     0
mais3_nmt                     0
mais3_aaam                    0
mais3_icdpic                  0
mais3_correct_nmt             0
mais3_correct_aaam            0
mais3_correct_icdpic          0
mais2_obs                     0
mais2_nmt                     0
mais2_aaam                    0
mais2_icdpic                  0
mais2_correct_nmt             0
mais2_co

In [35]:
res[res.iss_obs.isna()]

,inc_key,iss_obs,iss_nmt,iss_aaam,iss_icdpic,iss_correct_nmt,iss_correct_aaam,iss_correct_icdpic,iss16_obs,iss16_nmt,...,diff_reg_diff_sev_aaam,unmatched_obs_aaam,unmatched_pred_aaam,exact_icdpic,same_reg_same_sev_icdpic,same_reg_diff_sev_icdpic,diff_reg_same_sev_icdpic,diff_reg_diff_sev_icdpic,unmatched_obs_icdpic,unmatched_pred_icdpic
23,190027027754,NaN,NaN,18,41,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0
31,190027038612,NaN,9.0,8,6,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
34,190027038739,NaN,14.0,14,14,0,0,0,0,0,...,0.0,0.0,0.0,0.0,2.0,2.0,2.0,1.0,0.0,0.0
58,190027155991,NaN,NaN,10,8,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0
94,190027228272,NaN,10.0,5,2,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9668,190045125459,NaN,NaN,1,4,0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
9710,190045281222,NaN,NaN,5,4,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
9819,190045497568,NaN,NaN,43,45,0,0,0,0,0,...,1.0,0.0,3.0,0.0,4.0,5.0,10.0,1.0,0.0,4.0
9908,190045597551,NaN,NaN,1,6,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0


## Write out results

In [36]:
res.head()

,inc_key,iss_obs,iss_nmt,iss_aaam,iss_icdpic,iss_correct_nmt,iss_correct_aaam,iss_correct_icdpic,iss16_obs,iss16_nmt,...,diff_reg_diff_sev_aaam,unmatched_obs_aaam,unmatched_pred_aaam,exact_icdpic,same_reg_same_sev_icdpic,same_reg_diff_sev_icdpic,diff_reg_same_sev_icdpic,diff_reg_diff_sev_icdpic,unmatched_obs_icdpic,unmatched_pred_icdpic
0,190026915434,5.0,5.0,5,17,1,1,0,0,0,...,0.0,0.0,1.0,0.0,1.0,1.0,3.0,1.0,0.0,1.0
1,190026952586,5.0,5.0,5,3,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0
2,190026952733,9.0,9.0,25,9,1,0,1,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,190026952851,5.0,5.0,5,2,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0
4,190026953094,5.0,5.0,5,5,1,1,1,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0


In [37]:
res.to_csv(results_file, index=False)